# ドメイン特化

## この章で掴むこと

ドメイン特化では、モデル呼び出し以前に入力設計と評価設計を固める習慣を作ります。

前提: 確率的な予測モデルの見方と、深層学習の基礎が前提です。

到達目標: プロンプト、事前学習、微調整、RAG、効率化を実装視点で横断し、設計判断を言語化できる状態にします。

ここで扱う中心語は 「トークン」、「事前学習」、「微調整」、「RAG」、「推論最適化」、「ドメイン特化」 です。用語を先に暗記するのではなく、コード実行の結果と結びつけて理解します。

このノートは LLM 分野の初学者向けに、説明とコードを交互に読み進める設計です。最初から完璧に理解する必要はありません。大切なのは、各コードの目的を一文で言えることと、出力が変わる理由を自分で確かめることです。

## 実験 1: トークン近似を体験する

最初に、入力長とトークン量の関係を簡易計測します。コスト管理の第一歩です。

In [ ]:
text = '大規模言語モデルは文脈の与え方で応答品質が大きく変わる。'
char_len = len(text)
space_tokens = text.split()
rough_tokens = max(1, char_len // 2)
print('chars=', char_len, 'space_tokens=', len(space_tokens), 'rough_tokens=', rough_tokens)

厳密なトークン化ではありませんが、入力を短く保つ設計感覚を作るには十分です。

トークン を1つ変更して、出力が変わる原因を『入力・中間値・最終値』の順に追跡してください。

## 実験 2: プロンプトを構造化する

次に、指示・制約・出力形式を分離したテンプレートを作ります。曖昧さを減らすための実装技法です。

In [ ]:
instruction = '勾配降下法を初学者向けに説明する'
constraints = ['120字以内', '比喩は1つまで', '最後に要点を1行でまとめる']
prompt = f"指示: {instruction}\n制約: {'; '.join(constraints)}\n出力:"
print(prompt)
print('prompt_chars=', len(prompt))

この形にすると、失敗原因を特定しやすくなります。品質改善は原因分離のしやすさで決まります。

トークン を1つ変更して、出力が変わる原因を『入力・中間値・最終値』の順に追跡してください。

## 式と実装の往復

1. p_theta(x_t | x_<t)
2. L_CE = - sum_t log p_theta(x_t | x_<t)

## 実験 3: 検索文脈を結合する

ここで RAG の最小形を実装します。質問と関連文を結合し、回答入力を作る流れを確認します。

In [ ]:
question = 'ベルマン方程式を高校生向けに説明して'
retrieved = ['価値は将来報酬の割引和で定義する', '現在価値は次状態価値で再帰的に更新できる']
context = '\n'.join(f'- {c}' for c in retrieved)
final_input = f"質問:\n{question}\n\n参考文脈:\n{context}\n\n回答:"
print(final_input)

検索文脈を入れる目的は、モデルの記憶に頼りすぎないことです。根拠付き応答を作りやすくなります。

トークン を1つ変更して、出力が変わる原因を『入力・中間値・最終値』の順に追跡してください。

## 実験 4: 評価項目を数値化する

次に、回答を点検するための簡易スコアを定義します。評価軸を言語化すると改善が継続できます。

In [ ]:
answer = 'ベルマン方程式は、今の価値を次の価値で更新する再帰式です。'
checks = {'length_ok': len(answer) <= 120, 'has_keyword': '価値' in answer, 'has_recurrence': '再帰' in answer}
score = sum(1 for v in checks.values() if v) / len(checks)
print('checks=', checks)
print('score=', round(score, 3))

このような軽量評価でも、改善方向を揃える効果があります。実務ではこの評価軸をチームで共有します。

トークン を1つ変更して、出力が変わる原因を『入力・中間値・最終値』の順に追跡してください。

## 実験 5: 推論コストを見積もる

最後に、入力と出力の長さから概算コストを計算します。モデル選択は性能だけでなく費用とのバランスが必要です。

In [ ]:
input_tokens = 320
output_tokens = 180
price_per_1k = 0.0012
cost = (input_tokens + output_tokens) / 1000 * price_per_1k
print('estimated_cost=', round(cost, 6))

この見積もりを運用前に作ると、スケール時の予算超過を防ぎやすくなります。

トークン を1つ変更して、出力が変わる原因を『入力・中間値・最終値』の順に追跡してください。

## 振り返り

1. プロンプトだけで全問題を解決しようとする
2. 評価指標を決めずに改善を繰り返す
3. コストと品質のバランスを見ない

思考実験: 同じ質問に、文脈あり/なしで入力を作り、出力差の理由を分解します。

実装した処理を別データに移し替えて、壊れる点を先に探してください。

次は「軽量化（圧縮・最適化・効率化）」へ進み、今回のコードと何が変わるかを比較してください。

追加補足: LLM

説明密度が不足しているため、失敗例と再現手順を追記してください。
説明だけで終わらず、既存コードの入力値を2パターン変えて比較し、差分を記録すると理解が定着します。